In [1]:
import numpy as np
import tensorflow as tf

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)
  
def pk(actual, predicted, k=10):
    """
    Computes the precision at k.
    This function computes the prscision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    y_true= np.array(actual)
    onesmatrix=np.ones_like((actual))
    compare_matrix=np.dot(actual,onesmatrix)

    if not actual:
        return 0.0

    return np.mean(compare_matrix == predicted)

In [3]:
def mpk(actual, predicted, k=10):
    """
    Computes the mean precision at k.
    This function computes the mean precision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
            
            https://www.kaggle.com/wendykan/map-k-demo
    """
    return np.mean([pk(a,p,k) for a,p in zip(actual, predicted)])
  
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
            
            https://www.kaggle.com/wendykan/map-k-demo
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [8]:
actual = [1]
k=3
predicted = [1,1,3,4,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@5 =',apk(actual,predicted,k) )
print('PK@5 =', pk(actual,predicted,k) )

predicted = [2,1,3,4,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@5 =',apk(actual,predicted,k) )
print('PK@5 =', pk(actual,predicted,k) )

predicted = [3,2,1,4,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@5 =',apk(actual,predicted,k) )
print('PK@5 =', pk(actual,predicted,k) )

predicted = [4,2,3,1,5]
print('Answer=',actual,'predicted=',predicted)
print('AP@5 =',apk(actual,predicted,k) )
print('PK@5 =', pk(actual,predicted,k) )

predicted = [4,2,3,5,1]
print('Answer=',actual,'predicted=',predicted)
print('AP@5 =',apk(actual,predicted,k) )
print('PK@5 =', pk(actual,predicted,k) )

predicted = [4,2,3,5,1]
print('Answer=',actual,'predicted=',predicted)
print('PK@5 =',pk(actual,predicted,k) )


('Answer=', [1], 'predicted=', [1, 1, 3, 4, 5])
('AP@5 =', 1.0)
('PK@5 =', 0.6666666666666666)
('Answer=', [1], 'predicted=', [2, 1, 3, 4, 5])
('AP@5 =', 0.5)
('PK@5 =', 0.3333333333333333)
('Answer=', [1], 'predicted=', [3, 2, 1, 4, 5])
('AP@5 =', 0.3333333333333333)
('PK@5 =', 0.3333333333333333)
('Answer=', [1], 'predicted=', [4, 2, 3, 1, 5])
('AP@5 =', 0.0)
('PK@5 =', 0.0)
('Answer=', [1], 'predicted=', [4, 2, 3, 5, 1])
('AP@5 =', 0.0)
('PK@5 =', 0.0)
('Answer=', [1], 'predicted=', [4, 2, 3, 5, 1])
('PK@5 =', 0.0)


In [5]:
actual=[[1],[1],[1],[1],[1]]
predicted=[[1,1,3,4,1],[2,1,3,4,5],[3,2,1,4,5],[4,2,3,1,5],[4,2,3,5,1]]
print('MAP@5 =',mapk(actual, predicted, k=5))
print('MP@5 =',mpk(actual, predicted, k=5))

('MAP@5 =', 0.45666666666666667)
('MP@5 =', 0.27999999999999997)


https://github.com/ChenglongChen/tensorflow-LTR/blob/master/src/metrics.py

In [6]:
def dcg(predicted_order):
    i = np.log2(1. + np.arange(1,len(predicted_order)+1))
    l = np.array(predicted_order)
    return np.sum(l/i)


def ndcg(score, top_ten=True):
    end = 10 if top_ten else len(score)
    sorted_score = np.sort(score)[::-1]
    dcg_ = dcg(score[:end])
    if dcg_ == 0:
        return 0
    dcg_max = dcg(sorted_score[:end])
    return dcg_/dcg_max

In [7]:
predicted_order_ =[3,2,3,0,1,2,3,2]
print(dcg(predicted_order_))
print((ndcg(predicted_order_)))

8.492056442164959
0.935908621453514
